# YOLO: Rastreamento de Objetos e Análise de Tráfego com Supervision

In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv

In [ ]:
model = YOLO()

In [ ]:
video_path = 'vehicle-counting.mp4'
output_video_path = 'output_traffic_analysis_sv.mp4'

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Erro: Não foi possível abrir o vídeo em {video_path}")
    exit() # Encerrar se o vídeo não puder ser aberto

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"Resolução do vídeo: {width}x{height}, FPS: {fps}")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
print(f"Vídeo de saída configurado: {output_video_path}")

In [ ]:
# Define as classes de interesse (veículos)
# Obtenha os nomes das classes do modelo YOLO
class_names = model.names

# Mapeie os IDs das classes para os nomes que queremos rastrear
target_classes_ids = [
    class_id for class_id, name in class_names.items()
    if name in ['car', 'truck']
]
print(f"IDs das classes de veículos a serem rastreados: {target_classes_ids}")

# Definir a linha de contagem (LineZone)

line_start = sv.Point(0, (height // 3) * 2)
line_end = sv.Point(width, (height // 3) * 2)
line_zone = sv.LineZone(start=line_start, end=line_end)

# Anotadores do Supervision para visualização
# BoxAnnotator: desenha as bounding boxes e labels
box_annotator = sv.BoxAnnotator(
    thickness=2,
)
# LineZoneAnnotator: desenha a linha de contagem e os contadores
line_zone_annotator = sv.LineZoneAnnotator(
    thickness=2,
    text_thickness=2,
    text_scale=1
)

In [ ]:

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(source=frame, persist=True, conf=0.3, iou=0.5, verbose=False)

    detections = sv.Detections.from_ultralytics(results[0])

    # Atualizar a LineZone com as novas detecções para contagem
    line_zone.trigger(detections=detections)

    annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)

    # Anotar o frame com a linha de contagem e os contadores (in/out)
    annotated_frame = line_zone_annotator.annotate(
        frame=annotated_frame, line_counter=line_zone
    )

    cv2.imshow('Traffic Analysis', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break # Sair se 'q' for pressionado

    # Escrever o frame anotado no vídeo de saída
    out.write(annotated_frame)

In [ ]:
cap.release() # Liberar o objeto de captura
out.release() # Liberar o objeto de escrita de vídeo
cv2.destroyAllWindows() # Fechar todas as janelas

print("\nEstatísticas Finais de Contagem (com rastreamento):")
print(f"- Objetos que entraram na zona de contagem: {line_zone.in_count}")
print(f"- Objetos que saíram da zona de contagem: {line_zone.out_count}")